Download data for house mouse crosses from Ballinger, Mack et al. (https://doi.org/10.1073/pnas.2214614120).

In [ ]:
# ADD WGET FOR COUNTS
# ! wget -P ../../data/ballinger/ https://raw.githubusercontent.com/malballinger/BallingerMack_PNAS_2023/refs/heads/main/data/processed/BAT.MALE.COLD.categories.forplot.txt
# ! wget -P ../../data/ballinger/ https://raw.githubusercontent.com/malballinger/BallingerMack_PNAS_2023/refs/heads/main/data/processed/BAT.MALE.WARM.categories.forplot.txt
# ! wget -P ../../data/ballinger/ https://raw.githubusercontent.com/malballinger/BallingerMack_PNAS_2023/refs/heads/main/data/processed/Liver.MALE.COLD.categories.forplot.txt
# ! wget -P ../../data/ballinger/ https://raw.githubusercontent.com/malballinger/BallingerMack_PNAS_2023/refs/heads/main/data/processed/Liver.MALE.WARM.categories.forplot.txt

In [3]:
import pandas as pd
import numpy as np


/home/maria/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
# now read in the parents' raw counts
data_path = "../../data/ballinger"
all_parents_counts = pd.read_csv(f'{data_path}/BallingerPNAS_readcounts/all_parents_counts.txt',sep='\t')
all_parents_counts = all_parents_counts.set_index('gene')
all_parents_sample_info = pd.read_csv(f'{data_path}/BallingerPNAS_readcounts/all_parents_sample_info.txt',sep='\t')

BAT_ASEreads_coldwarm_femalemale = pd.read_csv(f'{data_path}/BallingerPNAS_readcounts/BAT_ASEreads_coldwarm_femalemale.txt',sep='\t')
BAT_ASEreads_coldwarm_femalemale = BAT_ASEreads_coldwarm_femalemale.set_index('gene')
liver_ASEreads_coldwarm_femalemale = pd.read_csv(f'{data_path}/BallingerPNAS_readcounts/liver_ASEreads_warmcold_malefemale.txt',sep='\t')
liver_ASEreads_coldwarm_femalemale = liver_ASEreads_coldwarm_femalemale.set_index('gene')

genes1 = np.sort ( pd.read_csv(f'{data_path}/BAT.MALE.COLD.categories.forplot.txt',sep=' ').iloc[:,0].values ) 
genes2 = np.sort ( pd.read_csv(f'{data_path}/BAT.MALE.WARM.categories.forplot.txt',sep=' ').iloc[:,0].values )
genes3 = np.sort ( pd.read_csv(f'{data_path}/Liver.MALE.COLD.categories.forplot.txt',sep=' ').iloc[:,0].values )
genes4 = np.sort ( pd.read_csv(f'{data_path}/Liver.MALE.WARM.categories.forplot.txt',sep=' ').iloc[:,0].values )
genes_overlap = list(set(genes1).intersection(set(genes2)).intersection(set(genes3)).intersection(set(genes4)) )


genes_to_fit = {'BAT_cold': genes1,
                'BAT_warm': genes2,
                'Liver_cold': genes3,
                'Liver_warm': genes4}



In [5]:
# create gene name file
gene_names_raw1 = pd.read_csv(f"{data_path}/gProfiler_gene_names_1.csv")
gene_names_raw = pd.read_csv(f"{data_path}/gProfiler_gene_names.csv")
# concatenate
gene_names_df = pd.concat((gene_names_raw1,gene_names_raw))
gene_names = np.sort(gene_names_df.name.values)

In [6]:
P1 = 'Brazil'
P2 = 'NewYork'



In [7]:
# create count matrix for BAT, cold -- only using male samples (6 per P1, P2, H1, H2)

tissue = 'BAT'
temp = 'Cold'
m_counts_ = np.ones((24,len(genes_to_fit[f'{tissue}_{temp.lower()}'])))


p1_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h2_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]

# add parental counts
for i,p1_ in enumerate(p1_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p1_}'].values

for i,p2_ in enumerate(p2_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p2_}'].values

# add hybrid counts
for i,h1_ in enumerate(h1_samples):
    m_counts_[12+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h1_}'].values

for i,h2_ in enumerate(h2_samples):
    m_counts_[18+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_{tissue}_{temp.lower()}_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [7]:
# create count matrix for BAT, warm -- only using male samples (6 per P1, P2, H1, H2)

tissue = 'BAT'
temp = 'Warm'
m_counts_ = np.ones((24,len(genes_to_fit[f'{tissue}_{temp.lower()}'])))


p1_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h2_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]

# add parental counts
for i,p1_ in enumerate(p1_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p1_}'].values

for i,p2_ in enumerate(p2_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p2_}'].values

# add hybrid counts
for i,h1_ in enumerate(h1_samples):
    m_counts_[12+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h1_}'].values

for i,h2_ in enumerate(h2_samples):
    m_counts_[18+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_{tissue}_{temp.lower()}_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [10]:
# create count matrix for Liver, warm -- only using male samples (6 per P1, P2, H1, H2)

tissue = 'Liver'
temp = 'Cold'
m_counts_ = np.ones((24,len(genes_to_fit[f'{tissue}_{temp.lower()}'])))


p1_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue.upper()) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue.upper()) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h2_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]

# add parental counts
for i,p1_ in enumerate(p1_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p1_}'].values

for i,p2_ in enumerate(p2_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p2_}'].values

# add hybrid counts
for i,h1_ in enumerate(h1_samples):
    m_counts_[12+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h1_}'].values

for i,h2_ in enumerate(h2_samples):
    m_counts_[18+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_{tissue}_{temp.lower()}_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [11]:
# create count matrix for Liver, warm -- only using male samples (6 per P1, P2, H1, H2)

tissue = 'Liver'
temp = 'Warm'
m_counts_ = np.ones((24,len(genes_to_fit[f'{tissue}_{temp.lower()}'])))


p1_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue.upper()) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == tissue.upper()) & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h2_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]

# add parental counts
for i,p1_ in enumerate(p1_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p1_}'].values

for i,p2_ in enumerate(p2_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{p2_}'].values

# add hybrid counts
for i,h1_ in enumerate(h1_samples):
    m_counts_[12+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h1_}'].values

for i,h2_ in enumerate(h2_samples):
    m_counts_[18+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'{tissue}_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_{tissue}_{temp.lower()}_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [8]:
# create and save design matrix for model C
p1_index = [True]*6 + [False]*18
p2_index = [False]*6 + [True]*6 + [False]*12
h1_index = [False]*12 + [True]*6 + [False]*6
h2_index = [False]*18 + [True]*6 

# model C
design_matrix_modelC = np.ones((len(p1_index),4))
design_matrix_modelC[p1_index,1] = 0
design_matrix_modelC[p1_index,3] = 0
design_matrix_modelC[p2_index,2] = 0
design_matrix_modelC[h1_index,1] = 0


design_matrix_modelC_no_cis = design_matrix_modelC.copy()[:,np.array([0,2,3])]
design_matrix_modelC_no_trans = design_matrix_modelC.copy()
design_matrix_modelC_no_trans[p2_index,2] = design_matrix_modelC_no_trans[p1_index,2]
design_matrix_modelC_no_trans[h1_index,2] = design_matrix_modelC_no_trans[h1_index,2]*2
design_matrix_modelC_no_trans[h2_index,2] = design_matrix_modelC_no_trans[h2_index,2]*2
design_matrix_modelC_no_trans = design_matrix_modelC_no_trans[:,:-1]
np.savetxt(f'{data_path}/male_design_model_C.txt',design_matrix_modelC, delimiter='\t', fmt='%d')
np.savetxt(f'{data_path}/male_design_model_C_no_cis.txt',design_matrix_modelC_no_cis, delimiter='\t', fmt='%d')
np.savetxt(f'{data_path}/male_design_model_C_no_trans.txt',design_matrix_modelC_no_trans, delimiter='\t', fmt='%d')

In [15]:
design_matrix_modelC_noint =  design_matrix_modelC[:,1:]
np.savetxt(f'{data_path}/male_design_model_C_noint.txt',design_matrix_modelC_noint, delimiter='\t', fmt='%d')

# null: no cis
design_matrix_modelC_noint_nocis = design_matrix_modelC_noint[:,1:]
np.savetxt(f'{data_path}/male_design_model_C_noint_nocis.txt',design_matrix_modelC_noint_nocis, delimiter='\t', fmt='%d')

# null: no trans
design_matrix_modelC_noint_notrans = design_matrix_modelC_noint[:,:1]
np.savetxt(f'{data_path}/male_design_model_C_noint_notrans.txt',design_matrix_modelC_noint_notrans, delimiter='\t', fmt='%d')

# Design and count matrices for more complex models

In [2]:
# model: tissue OR temp specific effects 

p1_index_tORt = np.array( [True]*12 + [False]*36 )
p2_index_tORt = np.array( [False]*12 + [True]*12 + [False]*24 )
h1_index_tORt = np.array( [False]*24 + [True]*12 + [False]*12 )
h2_index_tORt = np.array( [False]*36 + [True]*12 )

tissue_OR_temp_index = np.array( [True,True,True,True,True,True,False,False,False,False,False,False]*4 )


# model: tissue AND temperature specific effects
p1_index_tANDt = np.array( [True]*24 + [False]*72 ) 
p2_index_tANDt = np.array( [False]*24 + [True]*24 + [False]*48 )
h1_index_tANDt = np.array( [False]*48 + [True]*24 + [False]*24 )
h2_index_tANDt = np.array( [False]*72 + [True]*24 )

tissue_index = [[True]*12+[False]*12]*4
tissue_index = np.array( [item for sublist in tissue_index for item in sublist] )

temp_index = [[True]*6+[False]*6]*8
temp_index = np.array( [item for sublist in temp_index for item in sublist] )


# design matrix: model tORt no int
model_tORt_no_int = np.zeros((len(p1_index_tORt),7))
model_tORt_no_int[ np.ix_( np.where(p1_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,2],dtype=int) ) ] = 1   # weights for X_P1L
model_tORt_no_int[ np.ix_( np.where(p1_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,2,5],dtype=int) ) ] = 1   # weights for X_P1B 
model_tORt_no_int[ np.ix_( np.where(p2_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,1,3],dtype=int) ) ] = 1   # weights for X_P2L
model_tORt_no_int[ np.ix_( np.where(p2_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,1,3,4,6],dtype=int) ) ] = 1   # weights for X_P2B
model_tORt_no_int[ np.ix_( np.where(h1_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,2,3],dtype=int) ) ] = 1   # weights for X_H1L
model_tORt_no_int[ np.ix_( np.where(h1_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,2,3,5,6],dtype=int) ) ] = 1   # weights for X_H1B 
model_tORt_no_int[ np.ix_( np.where(h2_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,1,2,3],dtype=int) ) ] = 1   # weights for X_H2L
model_tORt_no_int[ np.ix_( np.where(h2_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,1,2,3,4,5,6],dtype=int) ) ] = 1   # weights for X_H2B

# with int
model_tORt_int = np.zeros((len(p1_index_tORt),8))
model_tORt_int[ np.ix_( np.where(p1_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,2],dtype=int) ) ] = 1   # weights for X_P1L
model_tORt_int[ np.ix_( np.where(p1_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,2,5,7],dtype=int) ) ] = 1   # weights for X_P1B 
model_tORt_int[ np.ix_( np.where(p2_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,1,3],dtype=int) ) ] = 1   # weights for X_P2L
model_tORt_int[ np.ix_( np.where(p2_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,1,3,4,6,7],dtype=int) ) ] = 1   # weights for X_P2B
model_tORt_int[ np.ix_( np.where(h1_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,2,3],dtype=int) ) ] = 1   # weights for X_H1L
model_tORt_int[ np.ix_( np.where(h1_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,2,3,5,6,7],dtype=int) ) ] = 1   # weights for X_H1B 
model_tORt_int[ np.ix_( np.where(h2_index_tORt & tissue_OR_temp_index)[0]  , np.array([0,1,2,3],dtype=int) ) ] = 1   # weights for X_H2L
model_tORt_int[ np.ix_( np.where(h2_index_tORt & ~tissue_OR_temp_index)[0]  , np.array([0,1,2,3,4,5,6,7],dtype=int) ) ] = 1   # weights for X_H2B


# design matrix: model tANDt no int
model_tANDt_no_int = np.zeros( (len(p1_index_tANDt),10) )
model_tANDt_no_int[ np.ix_( np.where(p1_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,2],dtype=int) ) ] = 1   # weights for X_P1LW
model_tANDt_no_int[ np.ix_( np.where(p1_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,2,8],dtype=int) ) ] = 1   # weights for X_P1LF
model_tANDt_no_int[ np.ix_( np.where(p1_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,2,5],dtype=int) ) ] = 1   # weights for X_P1BW
model_tANDt_no_int[ np.ix_( np.where(p1_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,2,5,8],dtype=int) ) ] = 1   # weights for X_P1BF

model_tANDt_no_int[ np.ix_( np.where(p2_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,1,3],dtype=int) ) ] = 1   # weights for X_P2LW
model_tANDt_no_int[ np.ix_( np.where(p2_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,1,3,7,9],dtype=int) ) ] = 1   # weights for X_P2LF
model_tANDt_no_int[ np.ix_( np.where(p2_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,1,3,4,6],dtype=int) ) ] = 1   # weights for X_P2BW
model_tANDt_no_int[ np.ix_( np.where(p2_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,1,3,4,6,7,9],dtype=int) ) ] = 1   # weights for X_P2BF

model_tANDt_no_int[ np.ix_( np.where(h1_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,2,3],dtype=int) ) ] = 1   # weights for X_H1LW
model_tANDt_no_int[ np.ix_( np.where(h1_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,2,3,8,9],dtype=int) ) ] = 1   # weights for X_H1LF
model_tANDt_no_int[ np.ix_( np.where(h1_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,2,3,5,6],dtype=int) ) ] = 1   # weights for X_H1BW
model_tANDt_no_int[ np.ix_( np.where(h1_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,2,3,5,6,8,9],dtype=int) ) ] = 1   # weights for X_H1BF

model_tANDt_no_int[ np.ix_( np.where(h2_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,1,2,3],dtype=int) ) ] = 1   # weights for X_H2LW
model_tANDt_no_int[ np.ix_( np.where(h2_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,1,2,3,7,8,9],dtype=int) ) ] = 1   # weights for X_H2LF
model_tANDt_no_int[ np.ix_( np.where(h2_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,1,2,3,4,5,6],dtype=int) ) ] = 1   # weights for X_H2BW
model_tANDt_no_int[ np.ix_( np.where(h2_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,1,2,3,4,5,6,7,8,9],dtype=int) ) ] = 1   # weights for X_H2BF


# with int
model_tANDt_int = np.zeros( (len(p1_index_tANDt),12) )
model_tANDt_int[ np.ix_( np.where(p1_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,2],dtype=int) ) ] = 1   # weights for X_P1LW
model_tANDt_int[ np.ix_( np.where(p1_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,2,8,11],dtype=int) ) ] = 1   # weights for X_P1LF
model_tANDt_int[ np.ix_( np.where(p1_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,2,5,10],dtype=int) ) ] = 1   # weights for X_P1BW
model_tANDt_int[ np.ix_( np.where(p1_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,2,5,8,10,11],dtype=int) ) ] = 1   # weights for X_P1BF

model_tANDt_int[ np.ix_( np.where(p2_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,1,3],dtype=int) ) ] = 1   # weights for X_P2LW
model_tANDt_int[ np.ix_( np.where(p2_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,1,3,7,9,11],dtype=int) ) ] = 1   # weights for X_P2LF
model_tANDt_int[ np.ix_( np.where(p2_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,1,3,4,6,10],dtype=int) ) ] = 1   # weights for X_P2BW
model_tANDt_int[ np.ix_( np.where(p2_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,1,3,4,6,7,9,10,11],dtype=int) ) ] = 1   # weights for X_P2BF

model_tANDt_int[ np.ix_( np.where(h1_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,2,3],dtype=int) ) ] = 1   # weights for X_H1LW
model_tANDt_int[ np.ix_( np.where(h1_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,2,3,8,9,11],dtype=int) ) ] = 1   # weights for X_H1LF
model_tANDt_int[ np.ix_( np.where(h1_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,2,3,5,6,10],dtype=int) ) ] = 1   # weights for X_H1BW
model_tANDt_int[ np.ix_( np.where(h1_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,2,3,5,6,8,9,10,11],dtype=int) ) ] = 1   # weights for X_H1BF

model_tANDt_int[ np.ix_( np.where(h2_index_tANDt & tissue_index & temp_index)[0]  , np.array([0,1,2,3],dtype=int) ) ] = 1   # weights for X_H2LW
model_tANDt_int[ np.ix_( np.where(h2_index_tANDt & tissue_index & ~temp_index)[0]  , np.array([0,1,2,3,7,8,9,11],dtype=int) ) ] = 1   # weights for X_H2LF
model_tANDt_int[ np.ix_( np.where(h2_index_tANDt & ~tissue_index & temp_index)[0]  , np.array([0,1,2,3,4,5,6,10],dtype=int) ) ] = 1   # weights for X_H2BW
model_tANDt_int[ np.ix_( np.where(h2_index_tANDt & ~tissue_index & ~temp_index)[0]  , np.array([0,1,2,3,4,5,6,7,8,9,10,11],dtype=int) ) ] = 1   # weights for X_H2BF


In [48]:
model_tANDt_int_FULL = np.zeros( (len(p1_index_tANDt),16) ) 
model_tANDt_int_FULL[:,:12] = model_tANDt_int
model_tANDt_int_FULL[:,12] = model_tANDt_int[:,10]*model_tANDt_int[:,11]  # beta_oe
model_tANDt_int_FULL[:,13] = model_tANDt_int[:,4]*model_tANDt_int[:,-1]  # beta_coe
model_tANDt_int_FULL[:,14] = model_tANDt_int[:,5]*model_tANDt_int[:,-1]  # beta_t1oe
model_tANDt_int_FULL[:,15] = model_tANDt_int[:,6]*model_tANDt_int[:,-1]  # beta_t2oe



In [30]:
model_noint_FULL = model_tANDt_int_FULL[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL.txt',model_noint_FULL, delimiter='\t', fmt='%d')

In [23]:
# and SAVE
np.savetxt(f'{data_path}/male_design_model_tORt_no_int.txt',model_tORt_no_int, delimiter='\t', fmt='%d')
np.savetxt(f'{data_path}/male_design_model_tORt_int.txt',model_tORt_int, delimiter='\t', fmt='%d')
np.savetxt(f'{data_path}/male_design_model_tANDt_no_int.txt',model_tANDt_no_int, delimiter='\t', fmt='%d')
np.savetxt(f'{data_path}/male_design_model_tANDt_int.txt',model_tANDt_int, delimiter='\t', fmt='%d')
# model_tANDt_int_FULL
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL.txt',model_tANDt_int_FULL, delimiter='\t', fmt='%d')

Now, create design matrices for specific hypotheses.

In [56]:
h2_index = h2_index_tANDt
# null: no cis effects at all -- GOOD
indices_to_keep = np.array([0,2,3,5,6,8,9,10,11,12,14,15])
design_no_cis = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_cis.txt',design_no_cis, delimiter='\t', fmt='%d')
indices_to_keep = np.array([2,3,5,6,8,9,10,11,12,14,15])
design_no_cis_noint = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_cis.txt',design_no_cis_noint, delimiter='\t', fmt='%d')

# null: no trans difference
design_no_trans = model_tANDt_int_FULL.copy()
indices_to_change = np.array([2,5,8,14])
indices_to_drop = np.array([3,6,9,15])
for col in indices_to_change:
    design_no_trans[p2_index_tANDt,col] = design_no_trans[p1_index_tANDt,col]
    design_no_trans[h1_index_tANDt,col] = design_no_trans[h1_index_tANDt,col]*2
    design_no_trans[h2_index_tANDt,col] = design_no_trans[h2_index_tANDt,col]*2
design_no_trans = np.delete(design_no_trans,indices_to_drop,1)
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_trans.txt',design_no_trans, delimiter='\t', fmt='%d')
design_noint_no_trans = design_no_trans[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_trans.txt',design_noint_no_trans, delimiter='\t', fmt='%d')


# null: no organ effect at all
indices_to_keep = np.array([0,1,2,3,7,8,9,11])
design_no_organ = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_organ.txt',design_no_organ, delimiter='\t', fmt='%d')
design_noint_no_organ = design_no_organ[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_organ.txt',design_noint_no_organ, delimiter='\t', fmt='%d')

# null: no temp effect at all
indices_to_keep = np.array([0,1,2,3,4,5,6,10])
design_no_temp = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_temp.txt',design_no_temp, delimiter='\t', fmt='%d')
design_noint_no_temp = design_no_temp[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_temp.txt',design_noint_no_temp, delimiter='\t', fmt='%d')

# null: no organ specific cis effect
indices_to_keep = np.array([0,1,2,3,5,6,7,8,9,10,11,12,14,15])
design_no_organ_cis = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_organ-cis.txt',design_no_organ_cis, delimiter='\t', fmt='%d')
design_noint_no_organ_cis = design_no_organ_cis[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_organ-cis.txt',design_noint_no_organ_cis, delimiter='\t', fmt='%d')


# null: no organ specific trans effect
design_no_organ_trans = model_tANDt_int_FULL.copy()
indices_to_change = np.array([5])
indices_to_drop = np.array([6])
for col in indices_to_change:
    design_no_organ_trans[p2_index_tANDt,col] = design_no_organ_trans[p1_index_tANDt,col]
    design_no_organ_trans[h1_index_tANDt,col] = design_no_organ_trans[h1_index_tANDt,col]*2
    design_no_organ_trans[h2_index_tANDt,col] = design_no_organ_trans[h2_index_tANDt,col]*2
design_no_organ_trans = np.delete(design_no_organ_trans,indices_to_drop,1)
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_organ-trans.txt',design_no_organ_trans, delimiter='\t', fmt='%d')
design_noint_no_organ_trans = design_no_organ_trans[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_organ-trans.txt',design_noint_no_organ_trans, delimiter='\t', fmt='%d')

# null: no temp specific cis effect
indices_to_keep = np.array([0,1,2,3,4,5,6,8,9,10,11,12,14,15])
design_no_temp_cis = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_temp-cis.txt',design_no_temp_cis, delimiter='\t', fmt='%d')
design_noint_no_temp_cis = design_no_temp_cis[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_temp-cis.txt',design_noint_no_temp_cis, delimiter='\t', fmt='%d')

# null: no temp specific trans effect
design_no_temp_trans = model_tANDt_int_FULL.copy()
indices_to_change = np.array([8])
indices_to_drop = np.array([9])
for col in indices_to_change:
    design_no_temp_trans[p2_index_tANDt,col] = design_no_temp_trans[p1_index_tANDt,col]
    design_no_temp_trans[h1_index_tANDt,col] = design_no_temp_trans[h1_index_tANDt,col]*2
    design_no_temp_trans[h2_index_tANDt,col] = design_no_temp_trans[h2_index_tANDt,col]*2
design_no_temp_trans = np.delete(design_no_temp_trans,indices_to_drop,1)
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_temp-trans.txt',design_no_temp_trans, delimiter='\t', fmt='%d')
design_noint_no_temp_trans = design_no_temp_trans[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_temp-trans.txt',design_noint_no_temp_trans, delimiter='\t', fmt='%d')

#null: no organ-temp specific effect at all
indices_to_keep = np.array([0,1,2,3,4,5,6,7,8,9,10,11,])
design_no_organ_temp_trans = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_organ-temp.txt',design_no_temp_trans, delimiter='\t', fmt='%d')
design_noint_no_organ_temp_trans = design_no_organ_temp_trans[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_organ-temp.txt',design_noint_no_temp_trans, delimiter='\t', fmt='%d')

# null: no organ-temp specific cis effect
indices_to_keep = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,14,15])
design_no_organ_temp_cis = model_tANDt_int_FULL[:,indices_to_keep].copy()
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_organ-temp-cis.txt',design_no_temp_cis, delimiter='\t', fmt='%d')
design_noint_no_organ_temp_cis = design_no_organ_temp_cis[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_organ-temp-cis.txt',design_noint_no_temp_cis, delimiter='\t', fmt='%d')

# null: no organ-temp specific trans effect
design_no_organ_temp_trans = model_tANDt_int_FULL.copy()
indices_to_change = np.array([14])
indices_to_drop = np.array([15])
for col in indices_to_change:
    design_no_organ_temp_trans[p2_index_tANDt,col] = design_no_organ_temp_trans[p1_index_tANDt,col]
    design_no_organ_temp_trans[h1_index_tANDt,col] = design_no_organ_temp_trans[h1_index_tANDt,col]*2
    design_no_organ_temp_trans[h2_index_tANDt,col] = design_no_organ_temp_trans[h2_index_tANDt,col]*2
design_no_organ_temp_trans = np.delete(design_no_organ_temp_trans,indices_to_drop,1)
np.savetxt(f'{data_path}/male_design_model_tANDt_int_FULL_no_organ-temp-trans.txt',design_no_temp_trans, delimiter='\t', fmt='%d')
design_noint_no_organ_temp_trans = design_no_organ_temp_trans[:,1:]
np.savetxt(f'{data_path}/male_design_model_tANDt_noint_FULL_no_organ-temp-trans.txt',design_noint_no_temp_trans, delimiter='\t', fmt='%d')

Now organize counts into X matrix.

In [14]:
# this time, only use genes that occur in all conditions
for key in genes_to_fit.keys():
    genes_to_fit[key] = np.sort(genes_overlap)

In [24]:
# create count matrix for liver AND BAT, warm
temp = 'Warm'
m_counts_ = np.ones((48,len(genes_to_fit[f'Liver_cold'])))


p1_liver_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_BAT_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_liver_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_BAT_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_liver_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h1_BAT_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h2_liver_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]
h2_BAT_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]

# add X_P1L
for i,p1_ in enumerate(p1_liver_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p1_}'].values

# add X_P1B
for i,p1_ in enumerate(p1_BAT_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p1_}'].values

# add X_P2L
for i,p2_ in enumerate(p2_liver_samples):
    m_counts_[12+i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p2_}'].values

# add X_P2B
for i,p2_ in enumerate(p2_BAT_samples):
    m_counts_[18+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p2_}'].values


# add X_H1L
for i,h1_ in enumerate(h1_liver_samples):
    m_counts_[24+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h1_}'].values

# add X_H1B
for i,h1_ in enumerate(h1_BAT_samples):
    m_counts_[30+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h1_}'].values

# add X_H2L
for i,h2_ in enumerate(h2_liver_samples):
    m_counts_[36+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h2_}'].values

# add X_H2B
for i,h2_ in enumerate(h2_BAT_samples):
    m_counts_[42+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_LiverBAT_warm_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [25]:
# create count matrix for liver AND BAT, cold
temp = 'Cold'
m_counts_ = np.ones((48,len(genes_to_fit[f'Liver_cold'])))


p1_liver_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_BAT_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_liver_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_BAT_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == temp) & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_liver_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h1_BAT_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno1' in s]
h2_liver_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]
h2_BAT_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'{temp.lower()}M_geno2' in s]

# add X_P1L
for i,p1_ in enumerate(p1_liver_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p1_}'].values

# add X_P1B
for i,p1_ in enumerate(p1_BAT_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p1_}'].values

# add X_P2L
for i,p2_ in enumerate(p2_liver_samples):
    m_counts_[12+i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p2_}'].values

# add X_P2B
for i,p2_ in enumerate(p2_BAT_samples):
    m_counts_[18+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p2_}'].values


# add X_H1L
for i,h1_ in enumerate(h1_liver_samples):
    m_counts_[24+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h1_}'].values

# add X_H1B
for i,h1_ in enumerate(h1_BAT_samples):
    m_counts_[30+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h1_}'].values

# add X_H2L
for i,h2_ in enumerate(h2_liver_samples):
    m_counts_[36+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h2_}'].values

# add X_H2B
for i,h2_ in enumerate(h2_BAT_samples):
    m_counts_[42+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_LiverBAT_cold_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [26]:
# create count matrix for warm and cold, liver
m_counts_ = np.ones((48,len(genes_to_fit[f'Liver_cold'])))


p1_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_warm_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'warmM_geno1' in s]
h1_cold_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'coldM_geno1' in s]
h2_warm_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'warmM_geno2' in s]
h2_cold_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'coldM_geno2' in s]

# add X_P1W
for i,p1_ in enumerate(p1_cold_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p1_}'].values

# add X_P1F
for i,p1_ in enumerate(p1_warm_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p1_}'].values

# add X_P2W
for i,p2_ in enumerate(p2_cold_samples):
    m_counts_[12+i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p2_}'].values

# add X_P2F
for i,p2_ in enumerate(p2_warm_samples):
    m_counts_[18+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p2_}'].values


# add X_H1W
for i,h1_ in enumerate(h1_warm_samples):
    m_counts_[24+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h1_}'].values

# add X_H1F
for i,h1_ in enumerate(h1_cold_samples):
    m_counts_[30+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h1_}'].values

# add X_H2W
for i,h2_ in enumerate(h2_warm_samples):
    m_counts_[36+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h2_}'].values

# add X_H2F
for i,h2_ in enumerate(h2_cold_samples):
    m_counts_[42+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_Liver_warmcold_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [27]:
# create count matrix for warm and cold, BAT
m_counts_ = np.ones((48,len(genes_to_fit[f'BAT_cold'])))


p1_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p2_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

h1_warm_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'warmM_geno1' in s]
h1_cold_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'coldM_geno1' in s]
h2_warm_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'warmM_geno2' in s]
h2_cold_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'coldM_geno2' in s]

# add X_P1W
for i,p1_ in enumerate(p1_cold_samples):
    m_counts_[i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p1_}'].values

# add X_P1F
for i,p1_ in enumerate(p1_warm_samples):
    m_counts_[6+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p1_}'].values

# add X_P2W
for i,p2_ in enumerate(p2_cold_samples):
    m_counts_[12+i,:] = all_parents_counts.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{p2_}'].values

# add X_P2F
for i,p2_ in enumerate(p2_warm_samples):
    m_counts_[18+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{p2_}'].values


# add X_H1W
for i,h1_ in enumerate(h1_warm_samples):
    m_counts_[24+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h1_}'].values

# add X_H1F
for i,h1_ in enumerate(h1_cold_samples):
    m_counts_[30+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h1_}'].values

# add X_H2W
for i,h2_ in enumerate(h2_warm_samples):
    m_counts_[36+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'Liver_{temp.lower()}'],f'{h2_}'].values

# add X_H2F
for i,h2_ in enumerate(h2_cold_samples):
    m_counts_[42+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_{temp.lower()}'],f'{h2_}'].values


# save 
np.savetxt(f'{data_path}/male_BAT_warmcold_X.txt',m_counts_, delimiter='\t', fmt='%d')

In [28]:
# FULL MODEL 
m_counts_ = np.ones((96,len(genes_to_fit[f'BAT_cold'])))


p1_liver_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_liver_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_BAT_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values

p1_BAT_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P1)]['Unnamed: 0'].values


p2_liver_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_liver_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'LIVER') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_BAT_warm_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Warm') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values

p2_BAT_cold_samples = all_parents_sample_info[ (all_parents_sample_info.sex=='M') & \
                         (all_parents_sample_info.condition == 'BAT') & \
                         (all_parents_sample_info.temperature == 'Cold') & \
                         (all_parents_sample_info.population == P2)]['Unnamed: 0'].values



h1_liver_warm_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'warmM_geno1' in s]
h1_liver_cold_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'coldM_geno1' in s]
h1_BAT_warm_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'warmM_geno1' in s]
h1_BAT_cold_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'coldM_geno1' in s]

h2_liver_warm_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'warmM_geno2' in s]
h2_liver_cold_samples = [s for s in liver_ASEreads_coldwarm_femalemale.columns if f'coldM_geno2' in s]
h2_BAT_warm_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'warmM_geno2' in s]
h2_BAT_cold_samples = [s for s in BAT_ASEreads_coldwarm_femalemale.columns if f'coldM_geno2' in s]






# add all parents
parent_samples = [p1_liver_warm_samples,p1_liver_cold_samples,p1_BAT_warm_samples,p1_BAT_cold_samples,
                  p2_liver_warm_samples,p2_liver_cold_samples,p2_BAT_warm_samples,p2_BAT_cold_samples]


n = 0
for parent_list in parent_samples:
    for i,p_ in enumerate(parent_list):
        m_counts_[n+i,:] = all_parents_counts.loc[genes_to_fit[f'BAT_cold'],f'{p_}'].values
    n+=6

# add all hybrid samples
hybrid_samples = [h1_liver_warm_samples,h1_liver_cold_samples,h1_BAT_warm_samples,h1_BAT_cold_samples,
                  h2_liver_warm_samples,h2_liver_cold_samples,h2_BAT_warm_samples,h2_BAT_cold_samples]

# add X_H1L
for h,hybrid_list in enumerate(hybrid_samples):
    for i,h_ in enumerate(hybrid_list):
        if h%4>=2:
            m_counts_[n+i,:] = BAT_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_cold'],f'{h_}'].values
        else:
            m_counts_[n+i,:] = liver_ASEreads_coldwarm_femalemale.loc[genes_to_fit[f'BAT_cold'],f'{h_}'].values
    n+=6

# save 
np.savetxt(f'{data_path}/male_LiverBAT_warmcold_X.txt',m_counts_, delimiter='\t', fmt='%d')


